In [1]:
import oracledb

connection = oracledb.connect(
    user="<your_user>", 
    password='<your_password>',
    dsn="<your_connection_string>"
)

cursor = connection.cursor()

In [9]:
# create tables

create_movie_table = """
    CREATE TABLE MOVIES (
        MOVIE_ID        NUMBER,
        TITLE           VARCHAR2(400),
        GENRES          JSON,
        SUMMARY         VARCHAR2(16000)
    )
"""

create_customer_table = """
    CREATE TABLE MOVIES_CUSTOMER (
        CUST_ID        NUMBER,
        FIRSTNAME      VARCHAR(200),
        LASTNAME       VARCHAR(200)
    )
"""

create_watched_table = """
    CREATE TABLE WATCHED_MOVIE (
        DAY_ID            TIMESTAMP(6),      
        MOVIE_ID          NUMBER,
        PROMO_CUST_ID     NUMBER
    )
"""

cursor.execute(create_movie_table)
cursor.execute(create_customer_table)
cursor.execute(create_watched_table)

In [10]:
cursor.execute("""
    INSERT INTO MOVIES (MOVIE_ID, TITLE, GENRES, SUMMARY) VALUES
        (1, 'Inception', '{"Action": "Sci-Fi"}', 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.'),
        (2, 'The Matrix', '{"Action": "Sci-Fi"}', 'A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.'),
        (3, 'The Godfather', '{"Drama": "Crime"}', 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'),
        (4, 'Titanic', '{"Romance": "Drama"}', 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'),
        (5, 'Toy Story', '{"Animation": "Adventure"}', 'A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy''s room.');
""")

cursor.execute("""
    INSERT INTO MOVIES_CUSTOMER (CUST_ID, FIRSTNAME, LASTNAME) VALUES
        (101, 'John', 'Doe'),
        (102, 'Jane', 'Smith'),
        (103, 'Sam', 'Wilson'),
        (104, 'Emily', 'Clark'),
        (105, 'Michael', 'Johnson')
""")

cursor.execute("""
    INSERT INTO WATCHED_MOVIE (DAY_ID, MOVIE_ID, PROMO_CUST_ID) VALUES
        (TO_TIMESTAMP('2024-10-30 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 1, 101),
        (TO_TIMESTAMP('2024-10-31 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 2, 101),
        (TO_TIMESTAMP('2024-09-30 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 3, 101),
        (TO_TIMESTAMP('2024-10-31 09:15:23.654321', 'YYYY-MM-DD HH24:MI:SS.FF'), 2, 102),
        (TO_TIMESTAMP('2024-11-01 16:45:12.987654', 'YYYY-MM-DD HH24:MI:SS.FF'), 3, 103),
        (TO_TIMESTAMP('2024-11-02 18:22:43.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 4, 104),
        (TO_TIMESTAMP('2024-11-03 20:01:00.000000', 'YYYY-MM-DD HH24:MI:SS.FF'), 5, 105)
""")

In [11]:
cpg = """
    CREATE OR REPLACE PROPERTY GRAPH CUSTOMER_WATCHED_MOVIES
        VERTEX TABLES (
            MOVIES_CUSTOMER AS CUSTOMER
                KEY(CUST_ID),
            MOVIES AS MOVIE
                KEY(MOVIE_ID)
        )
        EDGE TABLES(
            WATCHED_MOVIE AS WATCHED
                KEY(DAY_ID, MOVIE_ID, PROMO_CUST_ID)
                SOURCE KEY (PROMO_CUST_ID) REFERENCES CUSTOMER(CUST_ID)
                DESTINATION KEY (MOVIE_ID) REFERENCES MOVIE(MOVIE_ID)
        )
"""
cursor.execute(cpg)

In [12]:
sample_graph_query = """
    SELECT * FROM GRAPH_TABLE(CUSTOMER_WATCHED_MOVIES
        MATCH (c IS CUSTOMER) -[w IS WATCHED]-> (m IS MOVIE)
        COLUMNS(c.FIRSTNAME AS FIRSTNAME, c.LASTNAME AS LASTNAME, m.TITLE AS MOVIE_TITLE, w.DAY_ID as DAY_WATCHED)
    )
"""
for row in cursor.execute(sample_graph_query):
    print(row)

('John', 'Doe', 'Inception', datetime.datetime(2024, 10, 30, 12, 34, 56, 123456))
('John', 'Doe', 'The Matrix', datetime.datetime(2024, 10, 31, 12, 34, 56, 123456))
('John', 'Doe', 'The Godfather', datetime.datetime(2024, 9, 30, 12, 34, 56, 123456))
('Jane', 'Smith', 'The Matrix', datetime.datetime(2024, 10, 31, 9, 15, 23, 654321))
('Sam', 'Wilson', 'The Godfather', datetime.datetime(2024, 11, 1, 16, 45, 12, 987654))
('Emily', 'Clark', 'Titanic', datetime.datetime(2024, 11, 2, 18, 22, 43, 123456))
('Michael', 'Johnson', 'Toy Story', datetime.datetime(2024, 11, 3, 20, 1))


In [2]:
from langchain_openai import ChatOpenAI


# Choice 2, Set up OpenAI LLM
print ("WARNING: The step will fail if the API key is not present or is incorrect.")
print ("Please update the OpenAI_API_key before calling the llm the next step.")

# set the LLM to get response
llm = ChatOpenAI(
  model_name='gpt-3.5-turbo-16k',
  temperature = 0.1,
  openai_api_key="<your open AI API key>",
  max_tokens=2000
)
print("The LLM model you will use is OpenAI ChatGPT 3.5")

Please update the OpenAI_API_key before calling the llm the next step.
The LLM model you will use is OpenAI ChatGPT 3.5


In [13]:
from langchain_core.prompts import PromptTemplate

# Create a prompt template
template = "What is the genre of {movie} based on this summary: {summary}?"
prompt = PromptTemplate.from_template(template)

# Create a chain with the prompt and LLM
chain = prompt | llm

# Define your SQL query
sql = """
    SELECT DISTINCT MOVIE_TITLE, MOVIE_SUMMARY
    FROM GRAPH_TABLE( CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        COLUMNS (m.title as MOVIE_TITLE, m.summary as MOVIE_SUMMARY) 
    )
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print the results
for row in rows:
    # Run the chain and print the output
    result = chain.invoke({'movie': row[0], 'summary': row[1]})
    print(result.content)

The genre of Inception based on this summary is science fiction, specifically a heist thriller.
The genre of The Godfather based on this summary is crime/drama.
The genre of Toy Story based on this summary is animated comedy.
The genre of The Matrix based on this summary is science fiction.
The genre of Titanic based on this summary is romance.


In [15]:
# Create a prompt template
template = "Based on this dataset of movies a customer has watched, containing movie titles, genres and summaries, how would you describle the movie watching preferences of this user? {data}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

# Define your SQL query
sql = """
    SELECT DISTINCT MOVIE_TITLE, MOVIE_SUMMARY
    FROM GRAPH_TABLE( CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        WHERE c1.CUST_ID = 101
        COLUMNS (m.title as MOVIE_TITLE, m.summary as MOVIE_SUMMARY) 
    )
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the executed query
rows = cursor.fetchall()
# print(rows)

result = chain.invoke({'data': rows})
print(result.content)

Based on the dataset provided, the movie watching preferences of this user can be described as follows:

1. Genre Preference: The user seems to have a preference for movies that fall under the genres of science fiction, action, and crime. This is evident from their choice of movies like "Inception," "The Matrix," and "The Godfather," which all belong to these genres.

2. Complex and Intriguing Plots: The user appears to enjoy movies with complex and thought-provoking storylines. Both "Inception" and "The Matrix" involve intricate concepts and explore the nature of reality, while "The Godfather" delves into the complexities of organized crime and family dynamics.

3. Character-Driven Narratives: The user seems to appreciate movies that focus on well-developed characters and their personal journeys. "The Godfather" specifically revolves around the character development of the reluctant son as he takes over the family business.

4. Blend of Action and Drama: The user's movie choices indic

In [4]:
# Create a prompt template
template = "Based on this dataset of movies that customers have watched, containing customer IDs, movie IDs and date watched, Which customer has watched the most movies? {data}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

# Define your SQL query
sql = """
    SELECT *
    FROM GRAPH_TABLE( CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        WHERE c1.CUST_ID = 101
        COLUMNS (c1.CUST_ID as customer_id, m.MOVIE_ID as movie_id, e1.DAY_ID as date_watched) 
    )
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the executed query
rows = cursor.fetchall()
# print(rows)

result = chain.invoke({'data': rows})
print(result.content)

Based on the given dataset, customer 101 has watched the most movies.
